In [ ]:
# TODO: This notebook is broken. Fix it.

In [ ]:
from importlib import reload
from os.path import join
from collections import defaultdict
import sys
import os
import numpy as np
from matplotlib import pyplot
import cv2
cv2.setNumThreads(1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as tf
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
from datasets.dshdf5pose import Hdf5PoseDataset
import datatransformation
import neuralnets.models
import vis
import utils
import train

In [ ]:
net = neuralnets.models.LocalizerNet()
net.cuda()

In [ ]:
# Adjust as needed
path = join(os.environ['DATADIR'],'widerfacessingle.h5')

h, w = net.input_resolution
augh = round(h*1.2)
augw = round(w*1.2)

trainset = Hdf5PoseDataset(
        path,
        shuffle = True,
        subset = slice(500,None),
        transform=transforms.Compose([
            datatransformation.RescaleMaintainAspect((augh, augw)),
            datatransformation.RandomCrop(net.input_resolution),
            datatransformation.AdaptiveBrightnessContrastDistortion(),
            datatransformation.ImageColorDistort(),
            datatransformation.Flip(),
            datatransformation.Normalize(monochrome=True),
            datatransformation.ToTensor(),
]))


testset = Hdf5PoseDataset(
        path,
        shuffle = False,
        subset = slice(500),
        transform=transforms.Compose([
            datatransformation.RescaleMaintainAspect(net.input_resolution),
            datatransformation.Normalize(monochrome=True),
            datatransformation.ToTensor(),
]))

In [ ]:
train_loader = datatransformation.PostprocessingDataLoader(trainset, 
                          batch_size=32,
                          shuffle=False, 
                          num_workers=5,
                          postprocess = transforms.Compose([
                                datatransformation.BlurNoiseDistortion(),
                                datatransformation.MoveToGpu(),
                          ]))
test_loader = datatransformation.PostprocessingDataLoader(testset, 
                          batch_size=32,
                          shuffle=False, 
                          num_workers=5,
                          postprocess = datatransformation.MoveToGpu())

In [ ]:
def iterate_loader(loader):
    for batch in loader:
        for sample in utils.undo_collate(batch):
            yield vis.unnormalize_sample_to_numpy(sample)

%matplotlib notebook
vis.matplotlib_plot_iterable(iterate_loader(train_loader), vis.draw_dataset_sample)

In [ ]:
%matplotlib inline

In [ ]:
n_epochs = 100
optimizer = optim.Adam(net.parameters(), lr=1.e-3, weight_decay=1.e-5)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [60,90], gamma=0.1)

criterions = [
    train.Criterion('prob', train.LocalizerProbLoss(), 0.1, train=True, test=True),
    train.Criterion('box', train.LocalizerBoxLoss(), 1., train=True, test=True)
]

callbacks = [
    #train.SaveBestCallback(net, 'prob', model_dir='../model_files'),
    train.SaveBestCallback(net, 'box', model_dir='../model_files'),
]

train.run_the_training(
    n_epochs,
    optimizer,
    net,
    train_loader,
    test_loader,
    criterions,
    scheduler = scheduler,
    callbacks = callbacks)